In [322]:
import logging
import numpy as np
import math
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('AoC22')

# Part 1

In [323]:
logger.setLevel(logging.INFO)
with open('day9_input.txt') as f:
    contents = f.read().split('\n')

In [324]:
def separation_distance(x1,y1,x2,y2):
    d = ((x1-x2) ** 2 + (y1-y2) ** 2) ** 0.5
    return d

In [325]:
# First, calculate the grid size based on the moves and the max bounds
x_head, y_head = 0,0
x_max, y_max = 0,0
x_min, y_min = 0,0
for motion in contents:
    direction = motion.split(' ')[0]
    distance = int(motion.split(' ')[1])
    for move in range(0,distance):
        if direction == 'L':
            x_head -= 1
        elif direction == 'R':
            x_head += 1
        elif direction == 'U':
            y_head += 1
        elif direction == 'D':
            y_head -= 1
    x_max = max(x_max, x_head)
    y_max = max(y_max, y_head)
    x_min = min(x_min, x_head)
    y_min = min(y_min, y_head)
grid_width = x_max - x_min + 1
grid_height = y_max - y_min + 1

logger.info(f'{x_max = } {y_max = } {x_min = } {y_min = }')
logger.info(f'{grid_width = } {grid_height = }')

INFO:AoC22:x_max = 225 y_max = 16 x_min = -30 y_min = -249
INFO:AoC22:grid_width = 256 grid_height = 266


In [326]:
grid = np.full((grid_width,grid_height), 0)
# Set the starting positions to be offset by the minimum x and y to prevent negative coordinates for the grid array
x_head, y_head = abs(x_min), abs(y_min)
x_tail, y_tail = abs(x_min), abs(y_min)
grid[x_tail][y_tail] = 1
for motion in contents:
    direction = motion.split(' ')[0]
    distance = int(motion.split(' ')[1])
    for move in range(0,distance):
        if direction == 'L':
            x_head -= 1
        elif direction == 'R':
            x_head += 1
        elif direction == 'U':
            y_head += 1
        elif direction == 'D':
            y_head -= 1
        x_delta = x_head - x_tail
        y_delta = y_head - y_tail
        if separation_distance(x_head,y_head,x_tail,y_tail) > math.sqrt(2):
            # head and tail are not touching, therefore move the tail
            if x_delta == 0:
                # head and tail are in the same column => move the tail up or down
                y_tail += 1 if y_delta > 0 else -1
            elif y_delta == 0:
                # head and tail are in the same row => move the tail left or right
                x_tail += 1 if x_delta > 0 else -1
            else:
                # head and tail are diagonal => move the tail diagonal
                x_tail += 1 if x_delta > 0 else -1
                y_tail += 1 if y_delta > 0 else -1
        grid[x_tail][y_tail] = 1

In [327]:
grid.sum()

5619

# Part 2

In [328]:
# Create a dictionary of rope knots with the keys running from 0 to 9, with 0 representing the head.
# Each dictionary item is a list of [x,y] coordinates for the knot.
# The entire rope starts on the same grid location.
rope = dict()
for knot in range(0,10):
    rope[knot] = [abs(x_min),abs(y_min)]

In [329]:
# Move the rope head (0) and then check each following knot (1 through 9) to see if it should move.
# If it should move, use the direction logic from part one.
# After cycling along the entire rope, put a 1 in the grid for each knot position in the rope.
logger.setLevel(logging.CRITICAL)

grid = np.full((grid_width,grid_height), 0)
grid[rope[0][0]][rope[0][1]] = 1
move_count = 0
for motion in contents:
    direction = motion.split(' ')[0]
    distance = int(motion.split(' ')[1])
    logger.info(f'Move {direction} {distance}')
    for move in range(0,distance):
        logger.info(f'Rope before move: {rope}')
        move_count += 1

        # Move the head of the rope first
        if direction == 'L':
            rope[0][0] -= 1
        elif direction == 'R':
            rope[0][0] += 1
        elif direction == 'U':
            rope[0][1] += 1
        elif direction == 'D':
            rope[0][1] -= 1

        # After moving the head, run through the knots to see if they need to be moved
        for knot in range(1,10):
            x_delta = rope[knot-1][0] - rope[knot][0]
            y_delta = rope[knot-1][1] - rope[knot][1]

            logger.info(f'{move_count = }{knot = } {x_delta = } {y_delta = }')
            if separation_distance(rope[knot-1][0],rope[knot-1][1],rope[knot][0],rope[knot][1]) > math.sqrt(2):
                # knot and trailing knot are not touching, therefore move the trailing know
                logger.info(f'Need to move knot {knot = }')
                if x_delta == 0:
                    rope[knot][1] += 1 if y_delta > 0 else -1
                elif y_delta == 0:
                    rope[knot][0] += 1 if x_delta > 0 else -1
                else:
                    rope[knot][0] += 1 if x_delta > 0 else -1
                    rope[knot][1] += 1 if y_delta > 0 else -1
        logger.info(f'Rope after move: {rope}')
        # Finally, add the tail position to the grid
        grid[rope[9][0]][rope[9][1]] = 1
grid.sum()

2376